# Answering Business Questions Using SQL

In this project, we will use SQL to answer business questions. The database we will use is the Chinook database, a widely used database that represent a digital media store. You can find the database [here](https://github.com/lerocha/chinook-database?tab=readme-ov-file). 

Studying the schema diagram provided with the Chinook database is an important step before starting this project. You can find it in the ``` README ``` for the GitHub repository, a link for which is provided above.

## Setting Up SQL Environment and Linking the SQLite Database

In [17]:
%%capture
!pip install ipython-sql

In [18]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

## Data Overview

We will first study the columns in our database.

In [19]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
albums,table
sqlite_sequence,table
artists,table
customers,table
employees,table
genres,table
invoices,table
invoice_items,table
media_types,table
playlists,table


### Selecting New Albums to Purchase

The store needs to make a data-driven decision about which albums they should purchase. They need to know which albums will sell well in the United States. We will write a query to find the number and percentage of tracks sold in USA in different genres to determine which genres are the most popular.

For this, we will first create a temporary table called ``` genre_usa_tracks ``` to extract all the tracks sold in the US with their respective genres.

In [20]:
%%sql

WITH genre_usa_tracks AS
    (
        SELECT ii.InvoiceLineId, t.TrackId, g.Name AS Genre, i.BillingCountry
        FROM invoices AS i
        JOIN invoice_items AS ii ON i.InvoiceId = ii.InvoiceId
        JOIN tracks AS t ON ii.TrackId = t.TrackId
        JOIN genres AS g ON t.GenreId = g.GenreId
        WHERE i.BillingCountry = "USA"
)

SELECT Genre, 
COUNT(*) AS num_tracks_sold,
COUNT(*)/(SELECT CAST(COUNT(*) AS float) FROM genre_usa_tracks) * 100 AS percentage_tracks --cast to float because int division results in 0 (rounded off)
FROM genre_usa_tracks
GROUP BY Genre
ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


Genre,num_tracks_sold,percentage_tracks
Rock,157,31.781376518218625
Latin,91,18.421052631578945
Metal,64,12.955465587044534
Alternative & Punk,50,10.121457489878543
Jazz,22,4.4534412955465585
Blues,15,3.0364372469635628
TV Shows,14,2.834008097165992
R&B/Soul,12,2.42914979757085
Comedy,8,1.6194331983805668
Classical,8,1.6194331983805668


We can see that the top 2 genres with the most tracks sold in the US are Rock and Latin. The store should invest in albums from these genres, with a focus on Rock, since it has the highest percentage of tracks sold (nearly 32%). 

Metal and Alternative & Punk also show some potential, with track sales at approximately 12% and 10%, respectively.

## Analysing Employee Sales Performance

The store has a sales representative assigned to each of its customers. They need to find out which of their employees has managed to make the most sales. We will calculate the total dollars each salesperson has made in sales. To get a better idea of employees' sales performances, we will calculate the average amount of sales in dollars per customer.

The ``` HireDate ``` column is relevant to us since an employee's total sales depends on when they were hired.

In [21]:
%%sql

SELECT 
    e.FirstName || ' ' || e.LastName AS Name,
    e.HireDate,
    COUNT(c.CustomerId) AS customer_count,
    SUM(i.Total) AS total_sales_dollars,
    SUM(i.Total) / COUNT(c.CustomerId) AS avg_dollars_per_customer
FROM 
    employees AS e
JOIN 
    customers AS c 
    ON e.EmployeeId = c.SupportRepId
JOIN 
    invoices AS i 
    ON i.CustomerId = c.CustomerId
GROUP BY 1,2

 * sqlite:///chinook.db
Done.


Name,HireDate,customer_count,total_sales_dollars,avg_dollars_per_customer
Jane Peacock,2002-04-01 00:00:00,146,833.0400000000013,5.705753424657543
Margaret Park,2003-05-03 00:00:00,140,775.4000000000011,5.538571428571436
Steve Johnson,2003-10-17 00:00:00,126,720.160000000001,5.7155555555555635


We can see that Steve Johnson outperforms the rest of the employees. According to his hire date, he is the newest employee, yet still has the highest average dollars per customer of 5.72, which narrowly wins over Jane Peacock's 5.71 figure, who is the oldest employee.

## Analysing Sales by Country

The store needs to know how much dollars in sales they have made to customers from different countries. We will find the total number of customers and the amount of sales in dollars for each country.

- To clean up our analysis, we will group countries with 1 customer into an ``` Other ``` category.
- For better presentation, we will sort the results by sales amount, and then country name. ``` Other ``` will appear at the bottom.

In [22]:
%%sql

WITH 
    customer_counts AS (
        SELECT
            Country,
            COUNT(*) AS num_customers
        FROM
            customers
        GROUP BY 1
        ),

    labelled_invoices AS(
        SELECT 
            CASE WHEN cc.num_customers = 1 THEN 'Other'
            ELSE c.Country 
            END AS Country,
            i.CustomerId,
            i.invoiceId,
            ii.UnitPrice,
            ii.Quantity
        FROM
            customer_counts AS cc
        JOIN
            customers AS c USING(Country)
        JOIN 
            invoices AS i USING(CustomerId)
        JOIN
            invoice_items AS ii USING(InvoiceId)
        )

SELECT 
    Country,
    COUNT(DISTINCT CustomerId) AS Customers,
    SUM(UnitPrice * Quantity) AS total_sales_dollars
FROM 
    labelled_invoices
GROUP BY 1
ORDER BY
    CASE WHEN Country = 'Other' THEN 1 ELSE 0 END, 3 DESC

 * sqlite:///chinook.db
Done.


Country,Customers,total_sales_dollars
USA,13,523.060000000003
Canada,8,303.96000000000134
France,5,195.1000000000003
Brazil,5,190.10000000000028
Germany,4,156.48
United Kingdom,3,112.85999999999979
Czech Republic,2,90.23999999999997
Portugal,2,77.23999999999997
India,2,75.25999999999999
Other,15,604.3000000000038


## Albums vs. Individual Tracks 

The store wants to make a decision about purchasing entire albums from record stores or only selected tracks. They want to know if albums are more popular than individual tracks. We will help them make this decision by finding out how often customers buy whole albums instead of just individual tracks.

We need to find the number of invoices that include all the tracks from an entire album, indicating that a whole album was purchased. The invoice may list other tracks as well—we are only interested in seeing that it includes at least one complete album.

We will first create a table called ` invoice_first_tracks ` that displays the smallest ` TrackId ` from each invoice. Then we will find the album which this track belongs to, along with all the other tracks from the same album. 

We will use an `EXCEPT` statement to compare the tracks in the album to the tracks in the invoice:
- If the result of the `EXCEPT` is `NULL`, then the invoice contains all the album tracks and an album purchase was made.
- If the result of the `EXCEPT` is not `NULL`, then the invoice has at least one track missing from the album and an album purchase was not made.

In [23]:
%%sql

WITH invoice_first_tracks AS (
    SELECT 
        InvoiceId,
        MIN(TrackId) AS first_track_id
    FROM 
        invoice_items
    GROUP BY 
        1
)

SELECT
    album_purchase,
    COUNT(InvoiceId) AS number_of_invoices,
    CAST(COUNT(InvoiceId) AS FLOAT) / (SELECT COUNT(*) FROM invoices) * 100 AS percent_invoices
FROM
    (
    SELECT 
        ift.*,
        CASE
            WHEN (
                SELECT
                    t.TrackId
                FROM 
                    tracks t
                WHERE
                    AlbumId = (
                        SELECT 
                            t2.AlbumId
                        FROM
                            tracks t2
                        WHERE 
                            t2.TrackId = ift.first_track_id
                        )
                EXCEPT
                SELECT 
                    ii.TrackId
                FROM
                    invoice_items ii
                WHERE 
                    ii.InvoiceId = ift.InvoiceId
            ) IS NULL
            THEN
                "yes"
            ELSE
                "no"
            END AS album_purchase
    FROM
        invoice_first_tracks ift
    ) 
GROUP BY
    album_purchase -- group by album_purchase gives the number and percentage of invoices that were an album purchase

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent_invoices
no,400,97.0873786407767
yes,12,2.912621359223301


Album purchases make up only about 3% of purchases. It is clear that purchases of individual tracks are far more popular, and we recommend the store against buying complete albums, which would result in losing profits.